## Evaluation de nos Modeles ##


### Davies-Bouldin ###

On peut utiliser le coefficient de Davies-Bouldin qui cherche a avoir des clusters homogenes (serres) et separes (distants). Il est donne par la relation *D = max (T1+Tk)/(Sk,l)* avec T pour l'heterogeneite et S pour la separation. On veut donc T petit (numerateur) et S grand (denominateur). Donc plus le coefficient est **petit, mieux c'est**.

### Silouhette ###

On peut aussi utiliser le coefficient de silouhette qui etablit la probabilite qu'un point x soit bien classé. Est-il proche de son cluster (distance moyenne a tous les autres points du cluster a(x)) et quelle est la plus petite valeur de a(x) si il etait assigne a un autre cluster (b(x)). On obtient *s(x) = (b(x)-a(x)) / max(a,b)*. Il est compris entre -1 et 1 et on le veut **proche de 1**. Il utilisable avec "sklearn.metrics.silouhette_score".

### Stabilité ###

En lancant l'algorithme plusieurs fois sur les memes donnees mais avec une initialisation differente, un sous ensemble ou des donnees bruitees, a-t-on les memes resultats? Tres important pour choisir le **nombre de cluster**. 

### Connaissances specifiques du domaine ###

Par exemple si on classe des images pouvoir dire si elles sont classees selon le bon critere. Ca peut se voir a l'oeil facilement.

On peut aussi utiliser la comparaison de la *concordnace de deux partitions* du jeux de donnes (cf sklearn.metrics). Par exemple, on peut utiliser l'indice de Rand (plutot le ARI, **Adjusted Rand Index**) qui regarde si un point va etre classe de la meme maniere dans des partitions aleatoires des donees. Il est egale a 0 pour de l'aleatoire complet et a **1 quand le clustering correspond a la partition initiale**. On peut l'avoir par "sklearn.metrics.adjusted_rand_score".

## Optimisation des clusters ##

### Via le clustering Hierarchique ###

Pour bien separer les clusters sans prendre trop de temps de calcul, on peut faire du clustering hierarchique. On part d'un extreme (chaque point est un cluster ou tous les points sont dans un cluster) et on va graduellement vers l'autre extreme, la solution se trouvant forcement sur le chemin.

Le *clustering de Ward* minimise l'augmentation de la variance (ditance au carre) inter cluster. Avec d'autres methodes de clustering on peut aussi chercher a maximiser la distance entre les clusters. (lien simple, complet, centroidal, moyen)

Le clustering hierarchique se trouve dans le **module cluster de scikit-learn**.
- Avantages : nombre de clusters non definis a l'avance.
- Inconvenients : complexite algorithmique lourde.
Le clustering hierarchique est donc plus adapte pour les echantillons avec un faible nombres d'individus.

### Algorithme du k-means ###

On cherche ici a minimiser la variance intra cluster.

On utilise l'algorithme de Lloyd. Il converge rapidement mais peut etre dans un minimum local **donc** relancer plusieurs fois et evaluer la variance intra cluster a chaque fois pour comparer. Utilisable daans **sklearn.cluster.KMeans**. Par defaut l'algorithme est repete dix fois (n_init=10).

- Un probleme va etre que chaque point ayant le centroide le plus proche, on a que des clusters convexes (pas de croissant ou d'anneau) selon une tessellation de Voronoi. On peut palier grace a un kernel k-means utilisable via **sklearn.cluster.KernelMeans**.

- Un autre probleme va etre que l'initialisation se fait au hasard et on peut avoir des resultats tres differents et parfois tres mauvais. On va y pallier avec le k-means++ qui eparpille le plus possible les donnees avec les centroides initiaux. **sklearn.cluster.KMeans** se fait par defaut avec kmeans++ (parametre 'init').

### Partition avec DBSCAN (Density-Based Spatial Clustering of Applications with Noise) ###

On va utiliser le DBSCAN pour former des clusters non convexe (demi-lunes, cercles imbriques, etc). C'est un clustering par densite. On peut relier les points de proches en proches en restant dans le meme cluster.

- Avantages : Temps de calcul court sans predefinir le nombre de cluster, cluster de forme arbitraire possible.
- Inconvenients : difficile a utiliser en tres grande dimension (fleau de la dimensionnalite), choix des parametres epsilon et nmin delicat. (avoir assez de points interieurs) donc les clusters ont tous la meme densite. 

In [ ]:
# Algorithme du DBSCAN :

1. Prendre un point x qui n’a pas été visité

2. Construire N = voisinage(eps, x)

3.  If |N| < n_min:

        Marquer x comme bruit

    Else:

        Initaliser C = {x}
        agrandir_cluster(C, N, eps, n_min)

    Ajouter C à la liste des clusters
    Marquer tous les points de C comme visités

4. Repeat 1-3 until tous les points ont été visités

# Puis pour agrandir_cluster :

agrandir_cluster(C, N, eps, n_min):
    For u in N:
        If u n’a pas été visité:
            N’ = N(eps, u)
        If |N’| >= n_min:
            N = union(N, N’)
        If u n’appartient à aucun autre cluster:
            Ajouter u à C